In [1]:
import pandas as pd
from datetime import datetime
#!pip install quandl
import quandl

In [2]:
file = ['dow', 'euro_stoxx50', 'nasdaq', 'nikkei', 'oro', 'petroleo', 'shangai_stock_exchange', 'sp_500']
data = []
for i in file:
    data.append(pd.read_csv('./data/' + i + '.csv'))
file.insert(0, 'date')

In [3]:
for i in data:
    print(i.info(), '\n \n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1125 entries, 0 to 1124
Data columns (total 7 columns):
Fecha       1125 non-null object
Cierre      1125 non-null float64
Apertura    1125 non-null float64
Máximo      1125 non-null float64
Mínimo      1125 non-null float64
Vol.        1125 non-null object
% var.      1125 non-null object
dtypes: float64(4), object(3)
memory usage: 61.6+ KB
None 
 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1539 entries, 0 to 1538
Data columns (total 7 columns):
Fecha       1539 non-null object
Cierre      1539 non-null object
Apertura    1539 non-null object
Máximo      1539 non-null object
Mínimo      1539 non-null object
Vol.        1539 non-null object
% var.      1539 non-null object
dtypes: object(7)
memory usage: 84.2+ KB
None 
 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1512 entries, 0 to 1511
Data columns (total 7 columns):
Fecha       1512 non-null object
Cierre      1512 non-null float64
Apertura    1512 non-null float64
Máx

In [4]:
def price(data):
    try:
        amount = [float(i.replace(',','')) for i in data.Cierre]
        df = pd.DataFrame({'date': pd.to_datetime(data.Fecha),
                      'amount': amount})
    except:
        df = pd.DataFrame({'date':pd.to_datetime(data.Fecha),
                      'amount': data.Cierre.astype('float')})
    return df

def variance(data):
    var = [float(i.replace('%','')) for i in data.iloc[:,6]]
    df = pd.DataFrame({'date': pd.to_datetime(data.Fecha),
                      'var': var})
    return df

def generate_df(l, names, ind, rename = True):
    inner = pd.merge(l[0], l[1], how ='inner', on = 'date', left_on = None, right_on = None,
         left_index = True, right_index = True, sort = True)
    for i in range(2, len(l)):
        inner = pd.merge(inner, l[i], how ='inner', on = 'date', left_on = None, right_on = None,
                 left_index = True, right_index = True, sort = True)
    inner = pd.DataFrame(inner)
    if rename:
        inner.columns = names
        for i in range(1, len(names)):
            inner.rename(columns = {names[i]: names[i] + ind}, inplace = True)
    return inner.sort_values('date')

def get_info_bit(start, end, key):
    inf = quandl.get(key, start_date = start,end_date = end)
    return inf

In [5]:
amount = []
var = []
for i in data:
    amount.append(price(i))
    var.append(variance(i))

In [6]:
amount[0].head(5)

,date,amount
0,2020-12-31,55.50
1,2020-12-30,55.47
2,2020-12-29,54.33
3,2020-12-28,54.29
4,2020-12-24,54.79


In [7]:
var[0].head(5)

,date,var
0,2020-12-31,0.05
1,2020-12-30,2.10
2,2020-12-29,0.07
3,2020-12-28,-0.91
4,2020-12-24,0.07


In [8]:
for i in amount:
    print('max: ', i.date.max(), '\t min: ', i.date.min())

max:  2020-12-31 00:00:00 	 min:  2015-01-04 00:00:00
max:  2020-12-31 00:00:00 	 min:  2015-01-04 00:00:00
max:  2020-12-31 00:00:00 	 min:  2015-01-04 00:00:00
max:  2020-12-30 00:00:00 	 min:  2015-01-04 00:00:00
max:  2021-01-01 00:00:00 	 min:  2015-01-01 00:00:00
max:  2020-12-31 00:00:00 	 min:  2015-01-01 00:00:00
max:  2020-12-31 00:00:00 	 min:  2015-01-04 00:00:00
max:  2020-12-31 00:00:00 	 min:  2015-01-04 00:00:00


In [9]:
amount = generate_df(amount, file, '(USD)')
amount.head()

,date,dow(USD),euro_stoxx50(USD),nasdaq(USD),nikkei(USD),oro(USD),petroleo(USD),shangai_stock_exchange(USD),sp_500(USD)
1063,2015-01-04,47.90,3009.28,65.94,16819.24,1134.55,52.23,2.95,2136.73
1042,2015-01-05,51.69,3100.76,66.93,17177.21,1298.90,52.48,2.86,2163.26
1022,2015-01-06,52.26,3317.52,67.77,18360.54,1237.30,53.20,2.96,2246.19
1000,2015-01-07,51.39,3273.11,67.72,19364.67,1329.00,48.86,3.08,2275.32
957,2015-01-09,41.66,3472.58,70.82,19633.75,1329.90,48.66,3.03,2385.26


In [10]:
var = generate_df(var, file, '(%)')
var.head()

,date,dow(%),euro_stoxx50(%),nasdaq(%),nikkei(%),oro(%),petroleo(%),shangai_stock_exchange(%),sp_500(%)
1063,2015-01-04,-0.17,1.85,-0.72,0.50,-0.04,0.21,-0.24,-1.24
1042,2015-01-05,1.35,1.57,2.37,1.61,0.75,-0.08,0.00,1.11
1022,2015-01-06,0.36,0.08,0.24,0.47,-6.22,0.51,0.00,0.22
1000,2015-01-07,0.43,0.60,0.77,0.33,0.90,2.39,0.49,0.28
957,2015-01-09,-4.80,-0.26,0.16,0.15,0.05,-0.39,0.20,0.84


In [11]:
quandl.ApiConfig.api_key = "yRoEr9d428JVxLEqdo77"
start="2015-01-01"
end="2021-01-01"

In [12]:
bit = {'BCHAIN/TOTBC':'Total de bitcoins disponibles',
      'BCHAIN/MKPRU':'Precio del bitcoin en dolares',
      'BCHAIN/AVBLS':'Tamaño promedio del bloque',
      'BCHAIN/MIREV':'Recompensas por bloques de monedas y tarifas de transacción pagadas a los mineros',
      'BCHAIN/HRATE':'Número estimado de Tera de hashes por segundo que realizan todos los mineros',
      'BCHAIN/ATRCT':'Tiempo medio de confirmación de transacción',
      'BCHAIN/NTRBL':'Número de transacciones de Bitcoin por bloque',
      'BCHAIN/NADDU':'Número total de un Bitcoin único'}
bitcoin_var = bit.keys()
bitcoin_var

dict_keys(['BCHAIN/TOTBC', 'BCHAIN/MKPRU', 'BCHAIN/AVBLS', 'BCHAIN/MIREV', 'BCHAIN/HRATE', 'BCHAIN/ATRCT', 'BCHAIN/NTRBL', 'BCHAIN/NADDU'])

In [13]:
bitcoin_data = []
for i in bitcoin_var:
    bitcoin_data.append(get_info_bit(start, end, i).reset_index())
for i in range(0, len(bitcoin_data)):
    bitcoin_data[i].rename(columns = {'Date':'date'}, inplace = True)

In [14]:
bitcoin_var = list(bitcoin_var)
bitcoin_var.insert(0, 'date')

In [15]:
bitcoin_data[0].head(5)

,date,Value
0,2015-01-01,13674725.0
1,2015-01-02,13678725.0
2,2015-01-03,13682825.0
3,2015-01-04,13686975.0
4,2015-01-05,13691175.0


In [16]:
for i in bitcoin_data:
    print(i.info(), '\n \n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2192 entries, 0 to 2191
Data columns (total 2 columns):
date     2192 non-null datetime64[ns]
Value    2192 non-null float64
dtypes: datetime64[ns](1), float64(1)
memory usage: 34.3 KB
None 
 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2193 entries, 0 to 2192
Data columns (total 2 columns):
date     2193 non-null datetime64[ns]
Value    2193 non-null float64
dtypes: datetime64[ns](1), float64(1)
memory usage: 34.3 KB
None 
 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2193 entries, 0 to 2192
Data columns (total 2 columns):
date     2193 non-null datetime64[ns]
Value    2193 non-null float64
dtypes: datetime64[ns](1), float64(1)
memory usage: 34.3 KB
None 
 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2193 entries, 0 to 2192
Data columns (total 2 columns):
date     2193 non-null datetime64[ns]
Value    2193 non-null float64
dtypes: datetime64[ns](1), float64(1)
memory usage: 34.3 KB
None 
 

<class 'pandas.core.fram

In [17]:
bitcoin_data = generate_df(bitcoin_data, bitcoin_var, '')
bitcoin_data.rename(columns = {'BCHAIN/MKPRU':'BCHAIN/MKPRU (USD)', 'BCHAIN/MIREV':'BCHAIN/MIREV (USD)',
                   'BCHAIN/ATRCT':'BCHAIN/ATRCT (SEG)'}, inplace = True)
bitcoin_data.head(5)

,date,BCHAIN/TOTBC,BCHAIN/MKPRU (USD),BCHAIN/AVBLS,BCHAIN/MIREV (USD),BCHAIN/HRATE,BCHAIN/ATRCT (SEG),BCHAIN/NTRBL,BCHAIN/NADDU
0,2015-01-01,13674725.0,315.70,0.176213,1312680.60,335365.290092,7.150000,555.0,116012.0
1,2015-01-02,13678725.0,316.15,0.240704,1267761.50,323243.653101,6.933333,627.0,168563.0
2,2015-01-03,13682825.0,302.33,0.275661,1242878.63,331324.744428,6.433333,734.0,207571.0
3,2015-01-04,13686975.0,270.93,0.282322,1129778.10,335365.290092,8.033333,692.0,206228.0
4,2015-01-05,13691175.0,276.80,0.305495,1166988.80,339405.835756,6.816667,751.0,193982.0


In [18]:
file = ['eur_usd','cny_usd','jpy_usd','gbp_usd']
data = []
for i in file:
    data.append(pd.read_csv('./data/' + i + '.csv'))
file.insert(0, 'date')

In [20]:
for i in data:
    print(i.info(), '\n \n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Data columns (total 6 columns):
Fecha       1567 non-null object
Cierre      1567 non-null float64
Apertura    1567 non-null float64
Máximo      1567 non-null float64
Mínimo      1567 non-null float64
% var.      1567 non-null object
dtypes: float64(4), object(2)
memory usage: 73.5+ KB
None 
 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Data columns (total 6 columns):
Fecha       1567 non-null object
Cierre      1567 non-null float64
Apertura    1567 non-null float64
Máximo      1567 non-null float64
Mínimo      1567 non-null float64
% var.      1567 non-null object
dtypes: float64(4), object(2)
memory usage: 73.5+ KB
None 
 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 751 entries, 0 to 750
Data columns (total 6 columns):
Fecha       751 non-null object
Cierre      751 non-null float64
Apertura    751 non-null float64
Máximo      751 non-null float64
Mínimo      751 non-null 

In [ ]:
general = []
for i in data:
    general.append(price(i))
general = generate_df(general, file, '')
general.head()

In [ ]:
complete = [amount, bitcoin_data, general]
complete = generate_df(complete, file, '', False)
complete.head()

In [ ]:
complete.info()

In [ ]:
#complete.to_csv("data.csv", index = False)